## Categorical Feature Encoding Project - 머신러닝 편 (Baseline model)
### ML 베이스라인 모델 목차 
1. 변수 전처리(인코딩)
    - bin / ord : 데이터프레임 형태 
    - nom / day, month : 희소행렬 형태
    - 만들어진 두 타입의 데이터 이어 붙이기
2. 베이스라인 모델 학습 및 평가
    - 가벼운 Logistic Regression model 활용

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('train.csv', index_col = 'id')
test = pd.read_csv('test.csv', index_col = 'id')

display(train.head())
display(test.head())

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
300003,0,0,1,T,Y,Red,Star,Cat,China,Piano,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
300004,0,1,1,F,N,Red,Trapezoid,Dog,China,Piano,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [3]:
# 인코딩은 train과 test에 모두 해줄 것이므로 먼저 두개를 이어붙인다. target은 따로 빼서 저장
all_data = pd.concat([train, test], ignore_index = True)
all_data.drop(columns = 'target', axis = 1, inplace = True)
all_data.head(3)

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2


In [4]:
all_data.tail()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
499995,0,0,0,F,N,Green,Square,Lion,Canada,Theremin,...,9e4b23160,acc31291f,1,Novice,Lava Hot,j,A,Gb,1,3
499996,1,0,0,F,Y,Green,Trapezoid,Lion,China,Piano,...,cfbd87ed0,eae3446d0,1,Contributor,Lava Hot,f,S,Ed,2,2
499997,0,1,1,T,Y,Green,Trapezoid,Lion,Canada,Oboe,...,1108bcd6c,33dd3cf4b,1,Novice,Boiling Hot,g,V,TR,3,1
499998,1,0,0,T,Y,Blue,Star,Hamster,Costa Rica,Bassoon,...,606ac930b,d4cf587dd,2,Grandmaster,Boiling Hot,g,X,Ye,2,1
499999,0,0,0,T,Y,Green,Star,Lion,India,Piano,...,4ea576eb6,2d610f52c,2,Novice,Freezing,l,J,ex,2,2


In [5]:
print(all_data.shape)

(500000, 23)


In [6]:
# target
train_target = train['target']

## 1. 범주형 변수 인코딩
- `bin`: 그대로 사용하되, 0,1 값을 갖고 있지 않는 경우 변경
- `nom, day & month`: One-Hot 인코딩 (사이킷런 활용)
- `ord` : 순서가 있으므로 Label 인코딩 - 숫자 제외  
---
- 다만 여기서 주의점! 원-핫 인코딩 시키는 `nom(day, month 포함)`의 경우 열의 수가 매우 늘어날 것이므로 이를 DataFrame 형태로 변환하면 메모리에러가 난다..  
- 그래서 대안으로: `bin, ord` 변수 따로 / `nom` 따로 만들고
    - **`bin, ord` 변수는 데이터프레임에서 -> CSR 희소행렬 형태로 만들어 둘을 붙인다!**

### 1) 이진 변수(`bin_*`)
- bin_3, bin_4의 문자를 0, 1의 형태로 변경 

In [7]:
# bin_3, bin_4를 1,0 형태로 변경 
all_data['bin_3'] = all_data['bin_3'].apply(lambda x: 1 if x == 'T' else 0)
all_data['bin_4'] = all_data['bin_4'].apply(lambda x: 1 if x == 'Y' else 0)
all_data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,1,1,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,1,1,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,0,1,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,0,1,Red,Trapezoid,Snake,Canada,Oboe,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,0,0,Red,Trapezoid,Lion,Canada,Oboe,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


In [8]:
# bin_* 변수만 떼어내기
bin_cols = [f'bin_{i}' for i in range(0, 5)]
bin_data = all_data[bin_cols]
bin_data.head(3)

,bin_0,bin_1,bin_2,bin_3,bin_4
0,0,0,0,1,1
1,0,1,0,1,1
2,0,0,0,0,1


### 2) 순서형 변수(`ord_*`)
- 순서가 있으므로 Ordinal Encoding(Label Encoding)을 진행하여 순서를 숫자로 보존한다.
- 그 이후, 숫자값의 범위가 커지는 문제: 원-핫 인코딩을 또 해도 되지만 그러면 순위대로 보존되지 못함. => 스케일링!

In [9]:
# ord_* 변수만 떼어내기
ord_cols = [f'ord_{i}' for i in range(0, 6)] 
ord_data = all_data[ord_cols]
display(ord_data.head(3))

,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
0,2,Grandmaster,Cold,h,D,kr
1,1,Grandmaster,Hot,a,A,bF
2,1,Expert,Lava Hot,h,R,Jc


In [10]:
# 순서 설정이 필요한 ord_1, ord_2를 숫자값 순서로 설정해준다.
ord1_enc = {'Novice': 0, 'Contributor': 1, 'Expert': 2,
           'Master': 3, 'Grandmaster': 4}
ord2_enc = {'Freezing': 0 , 'Cold': 1, 'Warm': 2, 
           'Hot': 3, 'Boiling Hot': 4, 'Lava Hot': 5}

ord_data.loc[:, 'ord_1'] = ord_data.loc[:, 'ord_1'].map(ord1_enc)
ord_data.loc[:, 'ord_2'] = ord_data.loc[:, 'ord_2'].map(ord2_enc)
ord_data.head()

,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
0,2,4,1,h,D,kr
1,1,4,3,a,A,bF
2,1,2,5,h,R,Jc
3,1,4,4,i,D,kW
4,1,4,0,a,R,qP


In [11]:
# ord_3, ord_4, ord_5는 이미 알파벳이므로 labelEncoder로 숫자만 순서대로 부여한다.
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
ord_data['ord_3'] = encoder.fit_transform(ord_data['ord_3'].values)
ord_data['ord_4'] = encoder.fit_transform(ord_data['ord_4'].values)
ord_data['ord_5'] = encoder.fit_transform(ord_data['ord_5'].values)

ord_data.head()

,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
0,2,4,1,7,3,136
1,1,4,3,0,0,93
2,1,2,5,7,17,31
3,1,4,4,8,3,134
4,1,4,0,0,17,158


In [12]:
# 그런데 위처럼 진행하면 숫자의 스케일이 너무 넓어지므로, 정규분포를 따르도록 표준화 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
ord_data_scaled = scaler.fit_transform(ord_data)

pd.DataFrame(ord_data_scaled, columns = ord_data.columns).head()

,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
0,0.731992,1.416421,-0.660160,0.100961,-1.338303,0.757446
1,-0.672627,1.416421,0.324194,-1.726851,-1.748107,-0.018723
2,-0.672627,0.224377,1.308547,0.100961,0.574119,-1.137850
3,-0.672627,1.416421,0.816371,0.362077,-1.338303,0.721345
4,-0.672627,1.416421,-1.152336,-1.726851,0.574119,1.154555


In [13]:
# 앞에서 만든 bin_data와 col-bind로 붙이기 
all_data2 = pd.concat([bin_data, pd.DataFrame(ord_data_scaled, columns = ord_data.columns)], axis = 1)
all_data2

,bin_0,bin_1,bin_2,bin_3,bin_4,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5
0,0,0,0,1,1,0.731992,1.416421,-0.660160,0.100961,-1.338303,0.757446
1,0,1,0,1,1,-0.672627,1.416421,0.324194,-1.726851,-1.748107,-0.018723
2,0,0,0,0,1,-0.672627,0.224377,1.308547,0.100961,0.574119,-1.137850
3,0,1,0,0,1,-0.672627,1.416421,0.816371,0.362077,-1.338303,0.721345
4,0,0,0,0,0,-0.672627,1.416421,-1.152336,-1.726851,0.574119,1.154555
...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,0,0,0,-0.672627,-0.967667,1.308547,0.623193,-1.748107,-1.246153
499996,1,0,0,0,1,-0.672627,-0.371645,1.308547,-0.421271,0.710720,-1.390556
499997,0,1,1,1,1,-0.672627,-0.967667,0.816371,-0.160155,1.120525,-0.506085
499998,1,0,0,1,1,0.731992,1.416421,0.816371,-0.160155,1.393728,-0.217278


### ※ 이제 위에서 만든 `all_data2` 데이터와, 밑에서 만들 원-핫 인코딩된 `nom 희소행렬`을 붙일 것이다!

### 3) 명목형 변수(`nom_*`), 날짜 변수(`day, month`)
- nom_0 ~ nom_9 명목형 변수 및 날짜 변수에 One-Hot 인코딩 적용

In [14]:
# nom_* & 날짜 변수만 떼어내기
nom_cols = [f'nom_{i}' for i in range(0, 10)] 
nm_list = ['day', 'month']
nom_cols.extend(nm_list)
nom_data = all_data[nom_cols]
nom_data.head(3)

,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,day,month
0,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,2
1,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,7,8
2,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,7,2


In [15]:
# 변수가 엄청 늘어날 예정. 사이킷런의 OneHotEncoder 사용
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
nom_data_enc = encoder.fit_transform(nom_data)

In [16]:
# 희소행렬 생성 완료! (데이터프레임으로 출력은 불가)
nom_data_enc 

<500000x16295 sparse matrix of type '<class 'numpy.float64'>'
	with 6000000 stored elements in Compressed Sparse Row format>

### 4) all_data2, nom_data_enc 이어붙이기
- 데이터프레임 형태인 all_data2를 CSR 희소행렬 형태로 만들어서 붙인다.

In [17]:
from scipy import sparse
all_sprs = sparse.hstack([sparse.csr_matrix(all_data2), nom_data_enc],
                             format = 'csr')
all_sprs

<500000x16306 sparse matrix of type '<class 'numpy.float64'>'
	with 9959197 stored elements in Compressed Sparse Row format>

In [18]:
# all_sprs는 앞 부분은 train / 뒷 부분은 test 데이터이므로 분할
train_sprs = all_sprs[:len(train), :]
test_sprs = all_sprs[len(train):, :]
print(train_sprs.shape, test_sprs.shape)

(300000, 16306) (200000, 16306)


## 2. 베이스라인 모델 학습 및 평가
- 여기서는 베이스라인으로 로지스틱 회귀 모델을 선택하였다. 이진 분류에 사용하기 좋은 가벼운 모델이기 때문. (참고로 처음엔 DecisionTreeClassifier를 썼다가 매우 시간이 오래 걸림)
- 베이스라인 모델이기도 하고, 데이터의 수가 많기 때문에, k-fold를 쓰기보단 train / val set split을 하는 방식으로 성능을 평가해보자!

In [19]:
# train / val set split : target이 1과 0에 대해 갖고 있는 비율이 차이가 큰 불균형 데이터이므로 stratify를 설정해준다
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_sprs, train_target,
                                                  stratify = train_target, random_state = 99)
print(X_train.shape, X_val.shape)

(225000, 16306) (75000, 16306)


In [20]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state = 99)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)[:, 1]

# 성능 확인(AUC) - 주의점. predict_proba로 뽑힌 확률값으로 비교해야 함
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

0.7994263748583559

## 3. 베이스라인 모델로 제출해보기

In [21]:
sub = pd.read_csv('sample_submission.csv')
sub.head(3)

,id,target
0,300000,0.5
1,300001,0.5
2,300002,0.5


In [22]:
# 전체 데이터로 학습
model = LogisticRegression(random_state = 99, max_iter = 1000)
model.fit(train_sprs, train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=99, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# test data에 대해 예측 
y_pred = model.predict_proba(test_sprs)
y_pred[:, 1]

array([0.2986396 , 0.70010356, 0.07325867, ..., 0.38577974, 0.58019674,
       0.20617769])

In [24]:
# y_pred 중에서 두번째 열(즉, 1번 열)이 1로 예측될 확률이므로 이를 가져온다
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.head()

,0,1
0,0.701360,0.298640
1,0.299896,0.700104
2,0.926741,0.073259
3,0.596121,0.403879
4,0.094353,0.905647


In [25]:
del sub['target']
sub['target'] = y_pred[:, 1]
sub

,id,target
0,300000,0.298640
1,300001,0.700104
2,300002,0.073259
3,300003,0.403879
4,300004,0.905647
...,...,...
199995,499995,0.318023
199996,499996,0.121079
199997,499997,0.385780
199998,499998,0.580197


In [26]:
sub.to_csv('sub_sy_base.csv', index = False)

### test data에 대한 Score
- **public: 0.80231, private: 0.79764**
- 안 좋은 성능은 아니지만 리더보드를 보면 대부분 성능이 좋게 나오고 있어서 조금 더 성능을 올려보자!